In [1]:
pip install -r ../../requirements.txt


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [3]:
from zkstats.core import create_dummy, verifier_define_calculation, prover_gen_settings, setup, prover_gen_proof, verifier_verify, get_data_commitment_maps

In [4]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')

=======================  ZK-STATS FLOW =======================

In [5]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')
create_dummy(data_path, dummy_data_path)

In [18]:
# use large scale here to get the output really close to the value in dataset, which is important to Median.
# Be careful, since we may need to calculate average of 2 values in case of no middle value in dataset, 
# this means larger scale can still blow up circuit size, unlike Mode func that scale doesnt affect circuit size much.
scales = [7]
selected_columns = ['col_name']
commitment_maps = get_data_commitment_maps(data_path, scales)

In [19]:
# Verifier/ data consumer side: send desired calculation
from zkstats.computation import computation_to_model, State


def computation(s: State, data: list[torch.Tensor]) -> torch.Tensor:
    x = data[0]
    return s.median(x)

# error here in Median only matters in determining the median value in case it doesnt exist in dataset. (Avg of 2 middle values)
error = 0.01
_, verifier_model = computation_to_model(computation, error)

verifier_define_calculation(dummy_data_path, selected_columns, sel_dummy_data_path, verifier_model, verifier_model_path)

/Users/jernkun/Desktop/zk-stats-lib/zkstats/computation.py:166: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  is_precise_aggregated = torch.tensor(1.0)
/Users/jernkun/Library/Caches/pypoetry/virtualenvs/zkstats-OJpceffF-py3.11/lib/python3.11/site-packages/torch/onnx/symbolic_opset9.py:2174: FutureWarning: 'torch.onnx.symbolic_opset9._cast_Bool' is deprecated in version 2.0 and will be removed in the future. Please Avoid using this function and create a Cast node instead.
  return fn(g, to_cast_func(g, input, False), to_cast_func(g, other, False))


In [20]:
# Prover/ data owner side
_, prover_model = computation_to_model(computation, error)

prover_gen_settings(data_path, selected_columns, sel_data_path, prover_model,prover_model_path, scales, "resources", settings_path)

==== Generate & Calibrate Setting ====
scale:  [7]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":7,"param_scale":7,"scale_rebase_multiplier":10,"lookup_range":[-12494,12494],"logrows":15,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":7536,"total_const_size":7,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,7],"model_input_scales":[7],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":[{"GreaterThan":{"a":0.0}},"KroneckerDelta"],"check_mode":"UNSAFE","version":"7.0.0","num_blinding_factors":null}


In [21]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

spawning module 0
spawning module 2


==== setting up ezkl ====


spawning module 0
spawning module 2


Time setup: 3.290968894958496 seconds
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 49.546875
==== Generating Proof ====


spawning module 0
spawning module 2


proof:  {'instances': [[[12448964500226922122, 1569304359747940647, 1012944616373047521, 396517790474193570], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [12176007265991490802, 4416964430932312657, 15559879261920001877, 459064226310513766]]], 'proof': '21a7e6c14aeb411b44e2b37711400a2e74b94dc92189b445576b2b2ce4bb6b672c0f6ad955e52f26957b7516f462f5827f3824b0ce31453d15141a5dbc7607fb2b5d66316d6100f49b6060cc180c3a6ef226177e07c0825346093c5c3955339808e13868eb7b7108b0822c96317c538186fefa5653841feda935e8a76d5284d8118811eab3ec6354124e9465e3d3e7318826b7495977ee09ac60685c043a30af155b39990a3d38f668de0a9dcc3d6b3e90a31bede58ded3b3589c91045a489082d92f7b87ece917c52eb8a7077351ddd504b743b4768854d8ad6ae72cacaff0a1661247575220575456332b6b85429d2e07672f51bb171c69a681db8a384395e0799e6d49692e30fd40eb79cdf852dc5762f0f6458456901c7b72c25ea05f6390b8c5dd135fac0e61ae4713ec141d6a432a14ac1b360db10308d6babf1843e081d79b02eef2012e16f16e279355644d6dc3afbe0e9ec17f41c8c699d19cfbab7

In [22]:
# Verifier verifies
res = verifier_verify(proof_path, settings_path, vk_path, selected_columns, commitment_maps)
print("Verifier gets result:", res)

Verifier gets result: 49.546875
